## Trajectoryless Analysis Workshop Activity

In this notebook, we're running a sample simulation of 
a protein ([1HEL](https://www.rcsb.org/structure/1hel)) in water and analyzing it with live simulation streaming implemented in GROMACS and MDAnalysis.

We'll work up from the most basic IMDClient usage to a complex, live visualization of the simulation,
ending with a challenge to implement your own trajectoryless analysis code.

### 1. Starting the simulation

To run the simulation, first run the following commands in your terminal:
```bash
cd activity
# Preprocess the topology and run the simulation
./run.sh
```

Then, when you see the following line printed to the terminal, 
the simulation is ready for an IMD connection:

`IMD: Will wait until I have a connection and IMD_GO orders.`

### 2. Running analysis

Each analysis cell below connects to the running simulation and attempts to
analyze the entire trajectory. However, you can interrupt the cell at any time
for it to disconnect from the simulation so that another cell can be run.
Just press the interrupt button on the cell or on the jupyter toolbar.

### 3. Stopping the simulation

If you want to stop the simulation and restart it, you can also run CTRL+C in the terminal to stop the simulation entirely. The imd client will automatically determine that the stream is over and continue execution outside of the analysis loop.




### Creating an MDAnalysis Universe and running an analysis loop

The core object in MDAnalysis is a [Universe](https://userguide.mdanalysis.org/stable/universe.html),
which ties together the topology and trajectory. Using IMDClient, the trajectory argument is replaced
with a connection URL to the running simulation.

Here, we create a Universe from the simulation topology file, connect to the running simulation,
and print some data from the simulation.

**Ensure your simulation is running before running this (or any) cell. Before running the next cell, stop this cell or restart the simulation**

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:
    # Get an atom from the topology
    # Universe -> AtomGroup -> Atom
    atom = u.atoms[0]
    print("    time [         position         ] [         velocity         ] [           force          ] [            box           ]")

    # Analysis loop

    for ts in u.trajectory:
        print(f'{ts.time:8.3f} {atom.position} {atom.velocity} {atom.force} {u.dimensions[0:3]}')

except Exception as e:
    print(e)
finally:
    u.trajectory.close()

### Performing a simple distance calculation

Next, we'll calculate the distance between two atoms in the protein:
the alpha carbons of the n-terminus (residue 1) and the c-terminus (residue 129).

To learn more about MDAnalysis selections, see https://userguide.mdanalysis.org/stable/selections.html.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
import numpy as np

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:

    # Universe -> AtomGroup -> Atom
    nter = u.select_atoms("resid 1 and name CA").atoms[0]
    cter = u.select_atoms("resid 129 and name CA").atoms[0]


    for ts in u.trajectory:
        distance = np.sqrt(np.sum((nter.position - cter.position) ** 2))
        print(f'At time {ts.time:4.3f} ps, the n-terminus and c-terminus are {distance:4.3f} Angstroms apart', end='\r')

except Exception as e:
    print(e)
finally:
    u.trajectory.close()

## Live visualization of an analysis

Finally, we'll use matplot to create a live visualization of the distance between the two atoms.

In [ ]:
from imdclient.IMD import IMDReader
import numpy as np
import MDAnalysis as mda
from graph_utils import LiveTimeseriesGraph

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:

    nter = u.select_atoms("resid 1 and name CA").atoms[0]
    cter = u.select_atoms("resid 129 and name CA").atoms[0]

    graph = LiveTimeseriesGraph(
        time_window=1.0, # ps
        dt=0.010, # ps
        title='Distance vs. Time',
        y_label='Distance (Å)',
        legend_label='Nter-Cter',
    )


    for ts in u.trajectory:
        distance = np.sqrt(np.sum((nter.position - cter.position) ** 2))
        graph.update(ts.time, distance)

except Exception as e:
    print(e)
finally:
    u.trajectory.close()

## Writing trajectories to disk

Using IMDv3, it's easy to write a subselection of the trajectory to disk using MDAnalysis selections on the topology. In this example, we write the trajectory of the protein (without the water) to disk.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:

    sel = u.select_atoms("not resname SOL")

    desired_length = 10
    i = 0
    with mda.Writer("sample_simulation/protein.trr", sel.atoms.n_atoms) as w:
        for ts in u.trajectory:
            w.write(sel.atoms)
            i += 1
            
            if i == desired_length:
                break

except Exception as e:
    print(e)
finally:
    u.trajectory.close()

## Challenge 1

Your first challenge is to implement a new analysis function that calculates the **number of water molecules within 4 Angstroms of the protein during each timestep**.

Bonus points if you can create a live visualization of this data!

### Tips to get you started:

All water molecules in the simulation can be selected like this:
```python
sel = u.select_atoms('resname SOL')
```

All non-water (protein) molecules can be selected as well:
```python
sel = u.select_atoms('not resname SOL')
```

The number of atoms in the returned [AtomGroup](https://userguide.mdanalysis.org/stable/atomgroup.html) is selected as:
```python
len(sel)
```

Atoms can be selected based on a cutoff distance in Angstroms from another selection:
```python
sel = u.select_atoms('around 4 resname SOL')
```

Selections can be combined with 'and':
```python
# Not a useful selecton, just for example
sel = u.select_atoms('(resname SOL) and (not resname MET)')
```

Finally, the returned [AtomGroup](https://userguide.mdanalysis.org/stable/atomgroup.html) is statically-sized by default, with the selected atoms determined based on the first available frame.
To update the atom group for each frame, pass the `updating=True` key word argument:
```python
sel = u.select_atoms('around 4 resname SOL', updating=True)
```

See the [atom selection language](https://userguide.mdanalysis.org/stable/selections.html) page or ask in the chat for more!

Solutions are in `solutions.ipynb' if you are absolutely stuck.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
from graph_utils import LiveTimeseriesGraph

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:
    ## Graph setup (optional)

    for ts in u.trajectory:
        ## Your analysis code here

except Exception as e:
    print(e)
finally:
    u.trajectory.close()

## Challenge 2

Your next challenge is to find the radius of gyration of the protein for each frame of the simulation.
You do not need to find a separate radius of gyration for each axis. The
formula is calculated as:

$$R_{g} = \sqrt{ \frac{\sum_{i} m_ir_{i}^{2} }{ \sum_{i} m_i } }$$

where $r_i$ is the position of atom *i* relative to the center of mass of the protein and $m_i$ is its mass.

### Tips to get you started:

The center of mass of a protein can be calulated as:
```python
protein = u.select_atoms("not resname SOL")
center_of_mass = protein.center_of_mass()
```

The positions of each protein atom can be obtained in a numpy array of shape `(num_protein_atoms, 3)` as:
```python
pos = protein.positions
```

The masses of each protein atom can be obtained in a numpy array of shape `(num_protein_atoms)` as:
```python
masses = protein.masses
```

Finally, you can use numpy to calulate squares, square roots, and sums of numpy arrays:
```python
b = np.square(a)
d = np.sum(c)
f = np.sqrt(e)
```

This means you can calculate $r_i^2$ as:

```python
ri_sq = np.square((pos - center_of_mass))
```

Where the axes radii can be combined into a scalar radius like this:
```python
ri_sq = np.sum(ri_sq, axis=1)
```

For more inspiration, see the `radgyr` example method created in [writing your own trajectory analysis](https://userguide.mdanalysis.org/stable/examples/analysis/custom_trajectory_analysis.html)

Solutions are in `solutions.ipynb' if you are absolutely stuck.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
import numpy as np
from graph_utils import LiveTimeseriesGraph

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:

    ## Graph setup (optional)

    for ts in u.trajectory:
        ## Your analysis code here

except Exception as e:
    print(e)
finally:
    u.trajectory.close()